In [1]:

import numpy as np ; import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from seedtools  import connect
import pandas as pd
import fasttext
import re ,string,unicodedata,contractions
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
STOPWORDS =  stopwords.words("english")

In [2]:
def clean_text(text):
	text = str(text).lower()
	text = re.sub("[^\w\s]"," ",text)
	text =  ' '.join([w for w in text.split() if w not in STOPWORDS])
	return text

In [19]:
data   = pd.read_csv(connect("amazon_alexa.tsv"),sep="\t")[["verified_reviews","feedback"]]
data = data.dropna()
data["cleaned"] = data["verified_reviews"].apply(lambda  x : clean_text(x))
x_train,x_test,y_train,y_test =  train_test_split(data["cleaned"].values,data["feedback"].values,test_size=0.2,random_state=42)
x_train.shape

(2519,)

In [41]:


mapper = {"0":"negative","1":"positive"}


train = pd.DataFrame()
train['final_data'] = labeled_data = [
    f"__label__{mapper[str(y)]} {x}"
    for x, y in zip(x_train, y_train)
]

## old method :  "__label__"+y_train+" "+x_train

## data format : __label__positive thank you

test = pd.DataFrame()
test['final_data'] =  labeled_data = [
    f"__label__{mapper[str(y)]} {x}"
    for x, y in zip(x_test, y_test)
]


train.to_csv("train.csv",columns=['final_data'],index=False,header=False) ## compulsory
test.to_csv("test.csv",columns=['final_data'],index=False,header=False) ## compulsory


In [37]:
model  =  fasttext.train_supervised(input="train.csv")

In [39]:
n_samples_tested , precision, recall = model.test('test.csv')

print("No of samples tested : ",n_samples_tested)
print("Precision : ",precision)
print("Recall :",recall)

No of samples tested :  630
Precision :  0.9079365079365079
Recall : 0.9079365079365079
